install library

In [1]:
%pip install transformers
%pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


### Prune, calculate perplexity, and restore pruned layers

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import os
import pickle
import wandb
from transformers import AdamW, GPT2LMHeadModel, GPT2Tokenizer
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
import torch
import torch.nn.utils.prune as prune
from datasets import load_dataset
import numpy as np

# Path to save the sampled dataset
dataset_path = "/content/drive/MyDrive/HPML/Course_Project/sampled_dataset.pkl"

# Load or create the dataset
if os.path.exists(dataset_path):
    print("Loading existing sampled dataset...")
    with open(dataset_path, "rb") as f:
        full_subset = pickle.load(f)
else:
    print("Creating and saving a new sampled dataset...")
    dataset = load_dataset("wikitext", "wikitext-103-raw-v1", split="train")
    subset_ratio = 0.0025
    subset_size = int(len(dataset) * subset_ratio)
    full_subset = [dataset[i] for i in range(subset_size)]

    # Save the sampled dataset
    with open(dataset_path, "wb") as f:
        pickle.dump(full_subset, f)

from torch.utils.data import Dataset
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

full_subset = CustomDataset(full_subset)

# Split into train and test sets
train_size = int(0.8 * len(full_subset))
test_size = len(full_subset) - train_size
train_data, test_data = random_split(full_subset, [train_size, test_size])

# Load the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

# Path to save the fine-tuned model
fine_tuned_model_path = "/content/drive/MyDrive/HPML/Course_Project/fine_tuned_model"

# Fine-tune the original model if not already saved
if not os.path.exists(fine_tuned_model_path):
    print("Fine-tuning original model...")
    model = GPT2LMHeadModel.from_pretrained("gpt2")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.train()

    train_dataloader = DataLoader(train_data, batch_size=4, shuffle=True)
    optimizer = AdamW(model.parameters(), lr=5e-5)

    for epoch in range(5):  # Fine-tune for a few epochs
        epoch_loss = 0
        for batch in tqdm(train_dataloader):
            inputs = tokenizer(batch['text'], padding=True, truncation=True, return_tensors="pt")
            input_ids = inputs['input_ids']
            attention_mask = inputs['attention_mask']

            if input_ids.size(0) == 0 or input_ids.size(1) == 0:
                continue

            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
            loss = outputs.loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
        print(f"Epoch {epoch + 1}: Loss {epoch_loss / len(train_dataloader)}")

    model.save_pretrained(fine_tuned_model_path)
    tokenizer.save_pretrained(fine_tuned_model_path)
else:
    print("Fine-tuned model already exists. Skipping fine-tuning...")

# Define the W&B sweep configuration
sweep_config = {
    "method": "random",
    "metric": {
        "name": "perplexity",
        "goal": "minimize"
    },
    "parameters": {
        "first_ratio": {"values": [0.0, 0.1, 0.3, 0.5, 0.7, 0.9]},
        "middle_ratio": {"values": [0.0, 0.1, 0.3, 0.5, 0.7, 0.9]},
        "last_ratio": {"values": [0.0, 0.1, 0.3, 0.5, 0.7, 0.9]},
    }
}

def prune_mlp_layers(model, layer_count, ratios):
    first_end = layer_count // 3
    middle_start = first_end
    middle_end = 2 * layer_count // 3
    last_start = middle_end

    # Prune the first third of the layers
    for i in range(0, first_end):
        if ratios[0] > 0:
            prune.l1_unstructured(model.transformer.h[i].mlp.c_fc, name="weight", amount=ratios[0])
            prune.l1_unstructured(model.transformer.h[i].mlp.c_proj, name="weight", amount=ratios[0])

    # Prune the middle third of the layers
    for i in range(middle_start, middle_end):
        if ratios[1] > 0:
            prune.l1_unstructured(model.transformer.h[i].mlp.c_fc, name="weight", amount=ratios[1])
            prune.l1_unstructured(model.transformer.h[i].mlp.c_proj, name="weight", amount=ratios[1])

    # Prune the last third of the layers
    for i in range(last_start, layer_count):
        if ratios[2] > 0:
            prune.l1_unstructured(model.transformer.h[i].mlp.c_fc, name="weight", amount=ratios[2])
            prune.l1_unstructured(model.transformer.h[i].mlp.c_proj, name="weight", amount=ratios[2])

    # Remove pruning masks to finalize pruning
    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Linear):
            try:
                prune.remove(module, 'weight')
            except ValueError:
                pass

def evaluate_model(model, tokenizer, dataset, device):
    model.eval()
    dataloader = DataLoader(dataset, batch_size=8, shuffle=False)
    total_loss = 0.0
    total_tokens = 0
    total_correct = 0

    with torch.no_grad():
        for batch in dataloader:
            inputs = tokenizer(batch['text'], padding=True, truncation=True, return_tensors="pt")
            input_ids = inputs['input_ids'].to(device)
            attention_mask = inputs['attention_mask'].to(device)

            if input_ids.size(0) == 0 or input_ids.size(1) == 0:
                continue

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
            loss = outputs.loss
            total_loss += loss.item() * input_ids.numel()

            predictions = torch.argmax(outputs.logits, dim=-1)
            correct = (predictions == input_ids).float().sum().item()
            total_correct += correct
            total_tokens += input_ids.numel()

    perplexity = np.exp(total_loss / total_tokens) if total_tokens > 0 else float('inf')
    accuracy = total_correct / total_tokens if total_tokens > 0 else 0
    return perplexity, accuracy

def sweep_function():
    wandb.init()
    config = wandb.config

    # Load the fine-tuned model
    model = GPT2LMHeadModel.from_pretrained(fine_tuned_model_path)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    # Prune the MLP layers
    layer_count = len(model.transformer.h)
    prune_mlp_layers(model, layer_count, [config.first_ratio, config.middle_ratio, config.last_ratio])

    # Fine-tune the pruned model for a short time
    model.train()
    train_dataloader = DataLoader(train_data, batch_size=4, shuffle=True)
    optimizer = AdamW(model.parameters(), lr=5e-5)

    for epoch in range(1):
        epoch_loss = 0
        for batch in tqdm(train_dataloader, desc="Re-training after pruning"):
            inputs = tokenizer(batch['text'], padding=True, truncation=True, return_tensors="pt")
            input_ids = inputs['input_ids']
            attention_mask = inputs['attention_mask']

            if input_ids.size(0) == 0 or input_ids.size(1) == 0:
                continue

            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
            loss = outputs.loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
        wandb.log({"train_loss": epoch_loss / len(train_dataloader)})

    if len(test_data) > 0:
        first_test_sample = test_data[0]['text']
        print("First test sample text:\n", first_test_sample)
        # Tokenize it
        sample_inputs = tokenizer(first_test_sample, return_tensors="pt")
        print("First test sample tokenized input_ids:", sample_inputs['input_ids'])
        print("Decoded text from tokenized input:", tokenizer.decode(sample_inputs['input_ids'][0]))

    # Evaluate on test data
    perplexity, accuracy = evaluate_model(model, tokenizer, test_data, device)
    wandb.log({"perplexity": perplexity, "accuracy": accuracy})
    print(f"Test Perplexity: {perplexity:.4f}, Test Accuracy: {accuracy:.4f}")


# Initialize and run the sweep
sweep_id = wandb.sweep(sweep_config, project="gpt2-pruning-optimization")
wandb.agent(sweep_id, function=sweep_function)


Loading existing sampled dataset...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fine-tuned model already exists. Skipping fine-tuning...


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Create sweep with ID: wd1vpw3g
Sweep URL: https://wandb.ai/tokamak01823-new-york-university/gpt2-pruning-optimization/sweeps/wd1vpw3g


...

Re-training after pruning: 100%|██████████| 901/901 [03:21<00:00,  4.47it/s]


First test sample text:
  Hurricanes are an ever @-@ present threat during the summer and fall season , which puts Galveston in Coastal Windstorm Area . Galveston Island and the Bolivar Peninsula are generally at the greatest risk among the communities near the Galveston Bay . However , though the island and peninsula provide some shielding , the bay shoreline still faces significant danger from storm surge . 

First test sample tokenized input_ids: tensor([[39419,   389,   281,  1683,  2488,    12,    31,  1944,  2372,  1141,
           262,  3931,   290,  2121,  1622,   837,   543,  7584,  5027,  4223,
           261,   287, 43513,  3086, 12135,  9498,   764,  5027,  4223,   261,
          5451,   290,   262, 10797,   452,   283, 24078,   389,  4143,   379,
           262,  6000,  2526,  1871,   262,  5348,  1474,   262,  5027,  4223,
           261,  4696,   764,  2102,   837,   996,   262,  7022,   290, 28080,
          2148,   617, 43539,   837,   262, 15489, 15191,  1370,   991, 

accuracy,▁
perplexity,▁
train_loss,▁
accuracy,0.7317
perplexity,3.55781
train_loss,2.14224


wandb: Agent Starting Run: yanqd423 with config:
wandb: 	first_ratio: 0.3
wandb: 	last_ratio: 0.3
wandb: 	middle_ratio: 0.3


Re-training after pruning:  50%|█████     | 454/901 [01:43<01:38,  4.54it/s]

In [5]:
import os
import pickle
from transformers import AdamW, GPT2LMHeadModel, GPT2Tokenizer
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
import torch
import torch.nn.utils.prune as prune
from datasets import load_dataset
import numpy as np
from torch.utils.data import Dataset

# Constants
FIXED_FIRST_RATIO = 0.1

# Path to save the sampled dataset
dataset_path = "/content/drive/MyDrive/HPML/Course_Project/sampled_dataset.pkl"

class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

def evaluate_model(model, tokenizer, dataset, device):
    model.eval()
    dataloader = DataLoader(dataset, batch_size=8, shuffle=False)
    total_loss = 0.0
    total_tokens = 0

    with torch.no_grad():
        for batch in dataloader:
            inputs = tokenizer(batch['text'], padding=True, truncation=True, return_tensors="pt")
            input_ids = inputs['input_ids'].to(device)
            attention_mask = inputs['attention_mask'].to(device)

            if input_ids.size(0) == 0 or input_ids.size(1) == 0:
                continue

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
            loss = outputs.loss
            total_loss += loss.item() * input_ids.numel()
            total_tokens += input_ids.numel()

    perplexity = np.exp(total_loss / total_tokens) if total_tokens > 0 else float('inf')
    return perplexity

def prune_mlp_layers(model, layer_count, ratios):
    first_end = layer_count // 3
    middle_start = first_end
    middle_end = 2 * layer_count // 3
    last_start = middle_end

    for i in range(0, first_end):
        if ratios[0] > 0:
            prune.l1_unstructured(model.transformer.h[i].mlp.c_fc, name="weight", amount=ratios[0])
            prune.l1_unstructured(model.transformer.h[i].mlp.c_proj, name="weight", amount=ratios[0])

    for i in range(middle_start, middle_end):
        if ratios[1] > 0:
            prune.l1_unstructured(model.transformer.h[i].mlp.c_fc, name="weight", amount=ratios[1])
            prune.l1_unstructured(model.transformer.h[i].mlp.c_proj, name="weight", amount=ratios[1])

    for i in range(last_start, layer_count):
        if ratios[2] > 0:
            prune.l1_unstructured(model.transformer.h[i].mlp.c_fc, name="weight", amount=ratios[2])
            prune.l1_unstructured(model.transformer.h[i].mlp.c_proj, name="weight", amount=ratios[2])

    for name, module in model.named_modules():
        if isinstance(module, torch.nn.Linear):
            try:
                prune.remove(module, 'weight')
            except ValueError:
                pass

def fine_tune_model(model, train_data, tokenizer, device, epochs=3):
    model.train()
    dataloader = DataLoader(train_data, batch_size=4, shuffle=True)
    optimizer = AdamW(model.parameters(), lr=5e-5)

    for epoch in range(epochs):
        epoch_loss = 0
        for batch in tqdm(dataloader, desc=f"Fine-tuning Epoch {epoch + 1}/{epochs}"):
            inputs = tokenizer(batch['text'], padding=True, truncation=True, return_tensors="pt")
            input_ids = inputs['input_ids']
            attention_mask = inputs['attention_mask']

            if input_ids.size(0) == 0 or input_ids.size(1) == 0:
                continue

            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=input_ids)
            loss = outputs.loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
        print(f"Epoch {epoch + 1}/{epochs} Loss: {epoch_loss / len(dataloader):.4f}")

def test_ratios(ratio_combinations, num_trials):
    results = {}

    for ratios in ratio_combinations:
        print(f"Testing ratios: {ratios}")
        perplexities = []

        for trial in range(num_trials):
            print(f"Trial {trial + 1} for ratios {ratios}")

            # Load the fine-tuned model
            model = GPT2LMHeadModel.from_pretrained(fine_tuned_model_path)
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            model = model.to(device)

            # Prune the MLP layers
            layer_count = len(model.transformer.h)
            prune_mlp_layers(model, layer_count, ratios)

            # Fine-tune the model
            fine_tune_model(model, train_data, tokenizer, device, epochs=3)

            # Evaluate on test data
            perplexity = evaluate_model(model, tokenizer, test_data, device)
            perplexities.append(perplexity)

        # Calculate average perplexity
        avg_perplexity = sum(perplexities) / len(perplexities)
        results[tuple(ratios)] = avg_perplexity
        print(f"Average perplexity for ratios {ratios}: {avg_perplexity:.4f}")

    return results

# Main Execution
if __name__ == "__main__":
    # Define ratio combinations
    ratio_combinations = [
        [0.1, 0.2, 0.3],
        [0.1, 0.18, 0.32],
        [0.1, 0.3, 0.5],
        [0.1, 0.237, 0.563]
    ]

    # Number of trials per combination
    num_trials = 3

    # Load tokenizer
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    tokenizer.pad_token = tokenizer.eos_token

    # Load dataset
    if os.path.exists(dataset_path):
        print("Loading existing sampled dataset...")
        with open(dataset_path, "rb") as f:
            full_subset = pickle.load(f)
    else:
        print("Creating and saving a new sampled dataset...")
        dataset = load_dataset("wikitext", "wikitext-103-raw-v1", split="train")
        subset_ratio = 0.0025
        subset_size = int(len(dataset) * subset_ratio)
        full_subset = [dataset[i] for i in range(subset_size)]

        # Save the sampled dataset
        with open(dataset_path, "wb") as f:
            pickle.dump(full_subset, f)

    full_subset = CustomDataset(full_subset)

    # Split into train and test sets
    train_size = int(0.8 * len(full_subset))
    test_size = len(full_subset) - train_size
    train_data, test_data = random_split(full_subset, [train_size, test_size])

    # Fine-tuned model path
    fine_tuned_model_path = "/content/drive/MyDrive/HPML/Course_Project/fine_tuned_model"

    # Test ratio combinations
    results = test_ratios(ratio_combinations, num_trials)

    # Print final results
    print("\nFinal Results:")
    for ratios, avg_perplexity in results.items():
        print(f"Ratios {ratios}: Average Perplexity = {avg_perplexity:.4f}")


Loading existing sampled dataset...
Testing ratios: [0.1, 0.2, 0.3]
Trial 1 for ratios [0.1, 0.2, 0.3]


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Fine-tuning Epoch 1/3: 100%|██████████| 901/901 [03:12<00:00,  4.68it/s]


Epoch 1/3 Loss: 0.2044


Fine-tuning Epoch 2/3: 100%|██████████| 901/901 [03:20<00:00,  4.50it/s]


Epoch 2/3 Loss: 0.1742


...

Epoch 1/3 Loss: 0.3371


Fine-tuning Epoch 2/3: 100%|██████████| 901/901 [03:21<00:00,  4.48it/s]


Epoch 2/3 Loss: 0.2647


Fine-tuning Epoch 3/3: 100%|██████████| 901/901 [03:20<00:00,  4.49it/s]


Epoch 3/3 Loss: 0.2301
Average perplexity for ratios [0.1, 0.237, 0.563]: 1.1680

Final Results:
Ratios (0.1, 0.2, 0.3): Average Perplexity = 1.0972
Ratios (0.1, 0.18, 0.32): Average Perplexity = 1.0983
Ratios (0.1, 0.3, 0.5): Average Perplexity = 1.1427
Ratios (0.1, 0.237, 0.563): Average Perplexity = 1.1680
